In [ ]:
# 定义嵌入模型
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embed_model = OpenAIEmbeddings(
    api_key=os.getenv("BL_API_KEY"),
    base_url=os.getenv("BL_BASE_URL"),
    model="text-embedding-v3",
    dimensions=1024,
    check_embedding_ctx_length=False
)

In [ ]:
# 加载两个向量数据库
from langchain_milvus import Milvus, BM25BuiltInFunction

dense_vs = Milvus(
    collection_name='dense_hotpotqa_500',
    embedding_function=embed_model,
)

hybrid_vs = Milvus(
    collection_name='hybrid_hotpotqa_500',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)

In [ ]:
# 定义判断检索结果是否一致的函数
from typing import Tuple, List
from langchain_core.documents import Document

def is_same_result(
        dense_res: List[Tuple[Document, float]],
        hybrid_res: List[Tuple[Document, float]]
) -> bool:
    """
    比较两个向量数据库进行密集检索的结果是否一致

    Args:
        dense_res (List[Tuple[Document, float]]): 密集检索的结果
        hybrid_res (List[Tuple[Document, float]]): 混合检索的结果

    Returns:
        bool: 检索结果是否一致
    """
    # 检查两个结果的数量是否一致
    if len(dense_res) != len(hybrid_res):
        return False

    # 检查两个结果里面的具体内容是否一样
    for d_res, h_res in zip(dense_res, hybrid_res):
        if (
            d_res[1] != h_res[1]  # 比较分数
            or d_res[0].page_content != h_res[0].page_content  # 比较文档内容
        ):
            return False

    return True

In [ ]:
# 定义查询的 500 个问题
import json
from loguru import logger as log
import pprint

all_questions = []
question_path = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\evaluation\hotpotqa\hybrid_test_data_500.json'
with open(question_path, 'r', encoding='utf-8') as f:
    all_questions = json.load(f)

log.info(f'加载了 {len(all_questions)} 个问题')
pprint.pprint(all_questions[0])

# 定义查询的参数
top_k = 10
search_params = {
    'metric_type': 'L2',
    'params': {}
}

2025-07-23 17:17:43.267 | INFO     | __main__:<module>:11 - 加载了 500 个问题


{'_id': '5a8ba761554299240d9c2066',
 'answer': 'nearly 80 years',
 'context': [['Bus Stop (TV series)',
              ['Bus Stop is a 26-episode American drama which aired on ABC '
               'from October 1, 1961, until March 25, 1962, starring Marilyn '
               'Maxwell as Grace Sherwood, the owner of a bus station and '
               'diner in the fictitious town of Sunrise in the Colorado '
               'Rockies.',
               ' The program was adapted from William Inge\'s play, "Bus '
               'Stop", and Inge was a script consultant for the series, which '
               'followed the lives of travelers passing through the bus '
               'station and the diner.',
               " Maxwell's co-stars were Richard Anderson as District Attorney "
               'Glenn Wagner, Rhodes Reason as Sheriff Will Mayberry, Joan '
               'Freeman as waitress Elma Gahrigner, Bernard Kates as Ralph the '
               'coroner, and Buddy Ebsen as Virge Bles

In [ ]:
# 在两个向量数据库上使用 client.search 进行检索
from tqdm import tqdm

diff_res = []
same_res_num = 0

for question in tqdm(all_questions):
    raw_dense_res = dense_vs.client.search(
        collection_name='dense_hotpotqa_500',
        data=[embed_model.embed_query(question['question'])],
        anns_field='vector',  # 在 dense_hotpotqa_500 向量数据库中，密集嵌入的索引为 'vector'
        limit=top_k,
        search_params=search_params,
        output_fields=["*"]
    )
    dense_res = dense_vs._parse_documents_from_search_results(raw_dense_res)


    raw_hybrid_res = hybrid_vs.client.search(
        collection_name='hybrid_hotpotqa_500',
        data=[embed_model.embed_query(question['question'])],
        anns_field='dense',
        limit=top_k,
        search_params=search_params,
        output_fields=["*"]
    )
    hybrid_res = hybrid_vs._parse_documents_from_search_results(raw_hybrid_res)

    # 比较检索结果
    if is_same_result(dense_res, hybrid_res):
        same_res_num += 1
    else:
        tmp = {
            'question': question['question'],
            'dense_res': dense_res,
            'hybrid_res': hybrid_res
        }
        diff_res.append(tmp)

        # log.info(f'{question["question"]} 检索结果不一致')
        tqdm.write(f'{question["question"]} 检索结果不一致')
# 输出结果
log.info(f'检索结果一致的问题数量：{same_res_num} / {len(all_questions)}')
log.info(f'检索结果不一致的问题数量：{len(diff_res)}')

  0%|          | 1/500 [00:00<03:01,  2.75it/s]

How long did the career span of the actor who starred with Mickey Rooney and Marilyn Maxwell in Off Limits? 检索结果不一致


  2%|▏         | 10/500 [00:03<02:28,  3.30it/s]

Who was the running mate of the man about whose candidacy the 2016 book Insane Clown President: Dispatches from the 2016 Circus documents? 检索结果不一致


  3%|▎         | 15/500 [00:04<02:21,  3.42it/s]

...

 99%|█████████▊| 493/500 [02:25<00:02,  3.37it/s]

What is the estimated population of the city in which the suburb of Beaumont is located ? 检索结果不一致


100%|█████████▉| 499/500 [02:27<00:00,  3.09it/s]

From what county is Steven Avery, wrongfully convicted man on which the documentary Making a Murderer is based?  检索结果不一致


100%|██████████| 500/500 [02:27<00:00,  3.38it/s]
2025-07-23 17:24:09.526 | INFO     | __main__:<module>:43 - 检索结果一致的问题数量：314 / 500
2025-07-23 17:24:09.526 | INFO     | __main__:<module>:44 - 检索结果不一致的问题数量：186


In [ ]:
# 把检索不一致的内容持久化到文件中
res = []

for diff in diff_res:
    tmp = {
        'question': diff['question'],
    }

    d_res = diff['dense_res']
    h_res = diff['hybrid_res']

    i = 0
    for d_r, h_r in zip(d_res, h_res):
        tmp[str(i)] = {
            'dense_score': d_r[1],
            'hybrid_score': h_r[1],
            'dense_content': d_r[0].page_content,
            'hybrid_content': h_r[0].page_content
        }
        i += 1
    
    res.append(tmp)


res_path = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\client_search_diff.json'
os.makedirs(os.path.dirname(res_path), exist_ok=True)

with open(res_path, 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False)

In [ ]:
# 统计 500 个问题中，top_1 到 top_10 中不一样的分别有多少
from collections import defaultdict

res = defaultdict(int)
less_3 = 0

for diff in diff_res:
    d_res = diff['dense_res']
    h_res = diff['hybrid_res']

    i = 0
    for d_r, h_r in zip(d_res, h_res):
        if d_r[1] != h_r[1]:
            res[str(i)] += 1
            if i < 3:
                log.info(f'{i}: {diff['question']}')
                less_3 += 1
            break
        i += 1

2025-07-23 17:25:42.532 | INFO     | __main__:<module>:16 - 2: Spider9 was founded in 2011 by the head of which subsidiary of Wanxiang Group?
2025-07-23 17:25:42.533 | INFO     | __main__:<module>:16 - 0: ICI House is now named after the company that provides what type of item?
2025-07-23 17:25:42.533 | INFO     | __main__:<module>:16 - 2: The musician who did "Khalaara" was born on what date?
2025-07-23 17:25:42.534 | INFO     | __main__:<module>:16 - 2: When was the younger brother of Ervin Randle born?
2025-07-23 17:25:42.534 | INFO     | __main__:<module>:16 - 1: Swingin' Down Yonder is the first full-length, 12-inch album recorded by a singer known by what nickname?
2025-07-23 17:25:42.535 | INFO     | __main__:<module>:16 - 0: How many total staff (including part-time, flexible, and fixed contract staff) does the broadcaster, who released HyperNormalisation, employ?
2025-07-23 17:25:42.536 | INFO     | __main__:<module>:16 - 2: Are Harry Everett Smith and Vladimir Danilevich both

In [ ]:
res

defaultdict(int,
            {'4': 22,
             '9': 33,
             '8': 27,
             '2': 11,
             '7': 19,
             '5': 16,
             '0': 12,
             '3': 13,
             '6': 23,
             '1': 10})

### 结果总结

1. 在相同的问题和搜索参数下， `dense_vs.client.search` 和 `hybrid_vs.client.search` 的结果是不一致的

2. `dense_vs.similarity_search_with_score` 和 `hybrid_vs.client.search` 得到的不一致检索结果存储在 `diff.json` 中

    `dense_vs.client.search` 和 `hybrid_vs.client.search` 得到的不一致检索结果存储在 `client_search_diff.json` 中

    两个文件的 SHA-256 摘要是一致的，说明两次检索所捕获的不一致结果完全相同

In [ ]:
import hashlib
from pathlib import Path

BUF_SIZE = 1024 * 1024  # 1 MB 缓冲区，可按需调整

def sha256_of_file(path: Path) -> str:
    """返回文件的十六进制 SHA-256 摘要字符串。"""
    h = hashlib.sha256()
    with path.open("rb") as f:
        while chunk := f.read(BUF_SIZE):
            h.update(chunk)
    return h.hexdigest()

def files_equal_sha256(path1: Path, path2: Path) -> bool:
    """如果两个文件 SHA-256 摘要一致，返回 True"""
    return sha256_of_file(path1) == sha256_of_file(path2)

path1 = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\diff.json'
path2 = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\client_search_diff.json'
files_equal_sha256(Path(path1), Path(path2))

True